# CoVIS_19

Ich glaube es gibt genug Diskussionen zur aktuellen Krise, aber zu wenige Charts. In den letzten Tagen kursieren immer mehr Dashboards und Infografiken, leider sind diese oft recht statisch oder in Artikel eingebettet und verliert man den Artikel so verliert man auch die Grafik. Daher möchte ich:

####  1.) Die aktuellen Daten die von der Johns-Hopkins Universität herunterladen
####  2.) Die Daten anschauen und prozessieren
####  3.) Die Daten visualisieren

Das alles werde ich nicht aus dem Hut zaubern, ich benutze dazu die Programmiersprache F# und einige Bibliotheken die von der F# Community kostenfrei entwickelt wurden.

In [30]:
#load "Formatting_Extension.fsx"
open FSharp.Plotly
open System.IO    
open Deedle
open FSharp.Data
open Formatting_Extension

####  1.) Die aktuellen Daten die von der Johns-Hopkins Universität herunterladen

Nun können wir auch schon loslegen. Zunächst besorgen wir uns die Daten. 
Wie bereits angedeutet werden diese von der Johns-Hopkins Universität [zusammengetragen](https://github.com/CSSEGISandData/COVID-19) und sind freundlicherweise öffentlich einsehbar. den korrekten Link zu einer ihrer Tabellen habe ich bereits herausgesucht. Die Funktion "RequestString" besucht nun den link und läd die hinterlegten Daten herunter, dadurch sind wir immer auf dem aktuellen Stand wenn wir diese Zeile ausführen.

In [60]:
let rawData = Http.RequestString "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"

Wenn wir uns einen Teil der Daten mal anschauen, stellen wir fest, dass es sich um eine Komma separierte Textdatei handelt.

In [61]:
rawData.[..1000]

Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20
,Afghanistan,33,65,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74
,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123
,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60

####  2.) Die Daten anschauen und prozessieren
Damit lässt sich jetzt nicht so furchtbar gut arbeiten, also verwende ich die Bibliothek Deedle. Damit parse ich die Textdatei und bringe sie in ein Format das sich programatisch besser bearbeiten lässt. 

In [33]:
let bytes = System.Text.Encoding.UTF8.GetBytes rawData
let stream = new MemoryStream(bytes)
let parsedData = Frame.ReadCsv(stream)

Da die Daten nun als Objekt "Frame" vorliegen kann ich die Funktion "formatAsTable" benutzen und mir den Frame anschauen:

In [34]:
parsedData
|> formatAsTable 30 15
|> HtmlString

<!-- Plotly chart will be drawn inside this DIV -->

Wie man sehen kann schlüsselt die Tabelle die Zahl der infizierten nach Land und Datum auf. Da die Spalte "Province State" recht sparse ist, werde ich im folgenden diese Spalte und die Spalten "Lat" und Long" entfernen und die Fallzahlen tageweise aggregieren. Dazu gruppiere ich die Zeilen zunächst nach der Spalte "Country Region"

In [47]:
let aggregatedAndFiltered :Frame<string,_> = 
    parsedData    
    |> Frame.groupRowsBy "Country/Region"
    |> Frame.dropCol "Province/State"
    |> Frame.dropCol "Lat"
    |> Frame.dropCol "Long"
    |> Frame.dropCol "Country/Region"
    |> Frame.applyLevel fst Stats.sum 

aggregatedAndFiltered
|> formatAsTable 200 8
|> HtmlString 

<!-- Plotly chart will be drawn inside this DIV -->

Diese Tabelle enthält nun alle Länder, und die Fallzahlen pro Tag. Im Folgenden werde ich die Tabelle transponieren und ein paar Länder selektieren.

In [51]:
let timeCourses = 
    aggregatedAndFiltered.Transpose()
    |> Frame.sliceCols ["China";"Germany";"Italy";"US"]
    
timeCourses
|> formatAsTable 30 4
|> HtmlString 

<!-- Plotly chart will be drawn inside this DIV -->

####  3.) Die Daten visualisieren

In [62]:
let infectionTimeCourse = 
    timeCourses
    |> Frame.getNumericCols
    |> Series.sortByKey
    |> Series.map (fun c s ->
        Series.observations s
        |> Chart.Point
        |> Chart.withTraceName c
        )
    |> Series.values
    |> Chart.Combine
    |> Chart.withSize(1200.,500.)
    
infectionTimeCourse
|> toChartHTML
|> HtmlString

<!-- Plotly chart will be drawn inside this DIV -->

Nett! Zum Glück kann man das ganze auch animieren:

In [63]:
let infectionTimeCourses = 
    [|1 .. 3 .. timeCourses.RowCount|]
    |> Array.map (fun i -> 
        let ch = 
            timeCourses
            |> Frame.getNumericCols
            |> Series.sortByKey
            |> Series.map (fun c s ->
                s
                |> Series.take i
                |> Series.observations
                |> Chart.Point
                |> Chart.withTraceName c
                )
            |> Series.values
            |> Chart.Combine
            |> Chart.withSize(1200.,500.)
        ch |> toChartHTML |> HtmlString
    )
   
let chart = 
    infectionTimeCourses.[0]
    |> display
    
infectionTimeCourses
|> Seq.iter (fun ch -> 
     System.Threading.Thread.Sleep(4000)    
     chart.Update ch
     )

<!-- Plotly chart will be drawn inside this DIV -->